In [1]:
import os
import time
from collections import defaultdict
from shutil import copy, copytree, rmtree
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
from tensorflow.keras import backend as K, regularizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50

from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline

In [3]:
def getData():
  if "food-101" in os.listdir():
    print("data alrdy downloaded")
  else:
    print("downloading...")
    !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
    print("dataset downloaded!")
    !tar xzvf food-101.tar.gz
    print("extraction done!")

In [ ]:
# get food-101 data downloaded. you might need to enter huggingface secret key (HF_TOKEN) in environment variables.
getData()

### **we visualize images from various class, we use this in poster**

In [ ]:
numRows = 17
numCols = 6
figure, axes = plt.subplots(numRows, numCols, figsize=(25, 25))
figure.suptitle("Random images from each class", y=1.05, fontsize=22) # Adding y=1.05, fontsize=24 helped me fix the suptitle overlapping with axes issue
dataDir = "food-101/images/"
sortedFoods = sorted(os.listdir(dataDir))
foodId = 0
for i in range(numRows):
    for j in range(numCols):
        try:
            selectedFood = sortedFoods[foodId]
            foodId += 1
        except:
            break
        if selectedFood == '.DS_Store':
            continue
        selectedFoodImages = os.listdir(os.path.join(dataDir, selectedFood)) # returns the list of all files present in each food category
        randomSelectedFood = np.random.choice(selectedFoodImages) # picks one food item from the list as choice, takes a list and returns one random item
        image = plt.imread(os.path.join(dataDir, selectedFood, randomSelectedFood))
        axes[i][j].imshow(image)
        axes[i][j].set_title(selectedFood, pad=8)

plt.setp(axes, xticks=[], yticks=[])
plt.tight_layout()


In [11]:
def prepareData(filePath, source, destination):
    classImages = defaultdict(list)
    with open(filePath, 'r') as txt:
        paths = [line.strip() for line in txt.readlines()]
        for path in paths:
            food = path.split('/')
            classImages[food[0]].append(food[1] + '.jpg')

    for food in classImages.keys():
        print("\nCopying images into ", food)
        if not os.path.exists(os.path.join(destination, food)):
            os.makedirs(os.path.join(destination, food))
        for image in classImages[food]:
            copy(os.path.join(source, food, image), os.path.join(destination, food, image))

In [ ]:
# be careful with the file paths. i used what worked for me on google colab.
prepareData('/content/food-101/meta/train.txt', '/content/food-101/images', 'train')

In [ ]:
prepareData('/content/food-101/meta/test.txt', '/content/food-101/images', 'test')

In [31]:
# again go to root directory which has food-101 folder inside it
os.chdir('/content')

### **we create train_mini and test_mini for resnet50 with only 3 classes. later we will execute same code for 11 classes. and then later we will adapt this for inceptionv3 and store their train and test subsets in train_miniInception and test_miniInception.**


In [19]:
def createMiniDataset(foodItems, sourcePath, destinationPath):
  if os.path.exists(destinationPath):
    rmtree(destinationPath)
  os.makedirs(destinationPath)
  for item in foodItems:
    print("Copying images into", item)
    copytree(os.path.join(sourcePath, item), os.path.join(destinationPath, item))


In [29]:
foodItems = ['apple_pie', 'pizza', 'omelette']
sourceTrain = 'train'
destinationTrain = 'train_mini/'
sourceTest = 'test'
destinationTest = 'test_mini/'

In [ ]:

createMiniDataset(foodItems, sourceTrain, destinationTrain)

createMiniDataset(foodItems, sourceTest, destinationTest)

### **first, finetune ResNet50 on Food 101 dataset**

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

K.clear_session()
numClasses = 3
imageWidth, imageHeight = 224, 224
trainDataDir = 'train_mini'
validationDataDir = 'test_mini'
numTrainSamples = 2250
numValidationSamples = 750
batchSize = 16

trainDataGen = ImageDataGenerator(
    rescale=1. / 255,
    shearRange=0.2,
    zoomRange=0.2,
    horizontalFlip=True)

testDataGen = ImageDataGenerator(rescale=1. / 255)

trainGenerator = trainDataGen.flow_from_directory(
    trainDataDir,
    targetSize=(imageHeight, imageWidth),
    batchSize=batchSize,
    classMode='categorical')

validationGenerator = testDataGen.flow_from_directory(
    validationDataDir,
    targetSize=(imageHeight, imageWidth),
    batchSize=batchSize,
    classMode='categorical')

resNet50Model = ResNet50(weights='imagenet', include_top=False)
x = resNet50Model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(3, kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

model = Model(inputs=resNet50Model.input, outputs=predictions)
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
modelCheckpointer = ModelCheckpoint(
    filepath='best_model_3class.keras',
    verbose=1,
    save_best_only=True
)

csvLogger = CSVLogger('history_3class.log')

trainingHistory = model.fit(trainGenerator,
                            steps_per_epoch=numTrainSamples // batchSize,
                            validation_data=validationGenerator,
                            validation_steps=numValidationSamples // batchSize,
                            epochs=30,
                            verbose=1,
                            callbacks=[csvLogger, modelCheckpointer])

model.save('model_trained_3class.keras')

In [40]:
def plotAccuracy(history, title):
    plt.title(title)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    plt.show()

def plotLoss(history, title):
    plt.title(title)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'validation_loss'], loc='best')
    plt.show()

In [ ]:
plotAccuracy(trainingHistory,'Finetuned ResNet50 3 class Accuracy Curve')
plotLoss(trainingHistory,'Finetuned ResNet50 3 class Loss Curve')

### **Now, we finetune ResNet50 model with 11 random classes of data**

In [ ]:
n = 11
food_list = ['apple_pie', 'beef_carpaccio', 'bibimbap', 'cup_cakes', 'foie_gras', 'french_fries', 'garlic_bread', 'pizza', 'spring_rolls', 'spaghetti_carbonara', 'strawberry_shortcake']

In [ ]:

createMiniDataset(food_list, sourceTrain, destinationTest)
createMiniDataset(food_list, sourceTest, destinationTest)

In [ ]:
K.clear_session()

numClasses11 = n
imageWidth11, imageHeight11 = 224, 224
trainDataDir11 = 'train_mini'
validationDataDir11 = 'test_mini'
numTrainSamples11 = 8250
numValidationSamples11 = 2750
batchSize11 = 16

trainDataGen11 = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

testDataGen11 = ImageDataGenerator(rescale=1. / 255)

trainGenerator11 = trainDataGen11.flow_from_directory(
    trainDataDir11,
    target_size=(imageHeight11, imageWidth11),
    batch_size=batchSize11,
    class_mode='categorical')

validationGenerator11 = testDataGen11.flow_from_directory(
    validationDataDir11,
    target_size=(imageHeight11, imageWidth11),
    batch_size=batchSize11,
    class_mode='categorical')

resNet50Model11 = ResNet50(weights='imagenet', include_top=False)
x11 = resNet50Model11.output
x11 = GlobalAveragePooling2D()(x11)
x11 = Dense(128, activation='relu')(x11)
x11 = Dropout(0.2)(x11)

predictions11 = Dense(n, kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x11)

model11 = Model(inputs=resNet50Model11.input, outputs=predictions11)
model11.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer11 = ModelCheckpoint(filepath='best_model_11class.keras', verbose=1, save_best_only=True)
csvLogger11 = CSVLogger('history_11class.log')

history11Class = model11.fit(trainGenerator11,
                    steps_per_epoch=numTrainSamples11 // batchSize11,
                    validation_data=validationGenerator11,
                    validation_steps=numValidationSamples11 // batchSize11,
                    epochs=30,
                    verbose=1,
                    callbacks=[csvLogger11, checkpointer11])

model11.save('model_trained_11class.keras')

In [ ]:
plotAccuracy(history11Class,'Finetuned Resnet50 11 class Accuracy Curve')
plotLoss(history11Class,'Finetuned Resnet50 11 class Loss Curve')

In [48]:
foodItemsList = ['apple_pie', 'pizza', 'omelette']
sourceTrainPath = 'train'
destinationTrainPath = 'train_miniInception/'
sourceTestPath = 'test'
destinationTestPath = 'test_miniInception/'

In [ ]:
def createMiniDataset(foodItems, sourcePath, destinationPath):
  if os.path.exists(destinationPath):
    rmtree(destinationPath)
  os.makedirs(destinationPath)
  for item in foodItems:
    copytree(os.path.join(sourcePath, item), os.path.join(destinationPath, item))


In [ ]:

createMiniDataset(foodItemsList, sourceTrainPath, destinationTrainPath)
createMiniDataset(foodItemsList, sourceTestPath, destinationTestPath)

In [ ]:
K.clear_session()

numClassesInception = 3
imgWidthInception, imgHeightInception = 299, 299
trainDataDirInception = 'train_miniInception'
validationDataDirInception = 'test_miniInception'
numTrainSamplesInception = 2250
numValidationSamplesInception = 750
batchSizeInception = 16
epochsInception = 30

trainDataGenInception = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

testDataGenInception = ImageDataGenerator(rescale=1. / 255)

trainGeneratorInception = trainDataGenInception.flow_from_directory(
    trainDataDirInception,
    target_size=(imgHeightInception, imgWidthInception),
    batch_size=batchSizeInception,
    class_mode='categorical')

validationGeneratorInception = testDataGenInception.flow_from_directory(
    validationDataDirInception,
    target_size=(imgHeightInception, imgWidthInception),
    batch_size=batchSizeInception,
    class_mode='categorical')

baseModelInception = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(imgHeightInception, imgWidthInception, 3)
)

xInception = baseModelInception.output
xInception = GlobalAveragePooling2D()(xInception)
xInception = Dense(128, activation='relu')(xInception)
xInception = Dropout(0.2)(xInception)
predictionsInception = Dense(
    numClassesInception,
    activation='softmax',
    kernel_regularizer=regularizers.l2(0.005)
)(xInception)

modelInception = Model(inputs=baseModelInception.input, outputs=predictionsInception)

modelInception.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointerInception = ModelCheckpoint(
    filepath='best_inception_model_3class.keras',
    verbose=1,
    save_best_only=True
)
csvLoggerInception = CSVLogger('inception_history_3class.log')

historyInception = modelInception.fit(trainGeneratorInception,
                    steps_per_epoch=numTrainSamplesInception // batchSizeInception,
                    validation_data=validationGeneratorInception,
                    validation_steps=numValidationSamplesInception // batchSizeInception,
                    epochs=epochsInception,
                    verbose=1,
                    callbacks=[csvLoggerInception, checkpointerInception])

modelInception.save('inception_model_trained_3class.keras')

In [ ]:
plotAccuracy(historyInception, 'Finetuned InceptionV3 3 class Accuracy Curve')
plotLoss(historyInception, 'Finetuned InceptionV3 3 class Loss Curve')

In [ ]:
n = 11
food_list = ['apple_pie', 'beef_carpaccio', 'bibimbap', 'cup_cakes', 'foie_gras', 'french_fries', 'garlic_bread', 'pizza', 'spring_rolls', 'spaghetti_carbonara', 'strawberry_shortcake']

In [ ]:

createMiniDataset(food_list, sourceTrainPath, destinationTrainPath)
createMiniDataset(food_list, sourceTestPath, destinationTestPath)

In [ ]:
K.clear_session()
numClasses11 = n
imgWidth11, imgHeight11 = 299, 299
trainDataDir11 = 'train_miniInception'
validationDataDir11 = 'test_miniInception'
numTrainSamples11 = 8250 
numValidationSamples11 = 2750 
batchSize11 = 16
epochs11 = 30

trainDataGen11 = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

testDataGen11 = ImageDataGenerator(rescale=1. / 255)

trainGenerator11 = trainDataGen11.flow_from_directory(
    trainDataDir11,
    target_size=(imgHeight11, imgWidth11),
    batch_size=batchSize11,
    class_mode='categorical')

validationGenerator11 = testDataGen11.flow_from_directory(
    validationDataDir11,
    target_size=(imgHeight11, imgWidth11),
    batch_size=batchSize11,
    class_mode='categorical')

baseModel11 = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(imgHeight11, imgWidth11, 3)
)

x11 = baseModel11.output
x11 = GlobalAveragePooling2D()(x11)
x11 = Dense(128, activation='relu')(x11)
x11 = Dropout(0.2)(x11)
predictions11 = Dense(
    numClasses11,
    activation='softmax',
    kernel_regularizer=regularizers.l2(0.005)
)(x11)

model11 = Model(inputs=baseModel11.input, outputs=predictions11)

model11.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer11 = ModelCheckpoint(
    filepath='best_inception_model_11class.keras',
    verbose=1,
    save_best_only=True
)
csvLogger11 = CSVLogger('inception_history_11class.log')

history11Class = model11.fit(trainGenerator11,
                    steps_per_epoch=numTrainSamples11 // batchSize11,
                    validation_data=validationGenerator11,
                    validation_steps=numValidationSamples11 // batchSize11,
                    epochs=epochs11,
                    verbose=1,
                    callbacks=[csvLogger11, checkpointer11])

model11.save('inception_model_trained_11class.keras')

In [ ]:
plotAccuracy(history11Class,'Finetuned InceptionV3 11 class Accuracy Curve')
plotLoss(history11Class,'Finetuned InceptionV3 11 class Loss Curve')

# DONE HERE!


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def getTestGenerator(testDir, imgSize=(224,224), batchSize=16):
    testDataGen = ImageDataGenerator(rescale=1./255)
    return testDataGen.flow_from_directory(
        testDir,
        target_size=imgSize,
        batch_size=batchSize,
        class_mode='categorical',
        shuffle=False
    )

def evaluateModel(modelPath, testDir, imgSize=(224,224), batchSize=16):
    modelEval = load_model(modelPath)
    genEval = getTestGenerator(testDir, imgSize, batchSize)
    stepsEval = int(np.ceil(genEval.samples / batchSize))
    startEval = time.time()
    predsEval = modelEval.predict(genEval, steps=stepsEval, verbose=0)
    elapsedEval = time.time() - startEval

    yTrueEval = genEval.classes
    yPredEval = predsEval.argmax(axis=1)
    labelsEval = list(genEval.class_indices.keys())

    print(f"\n>>> Model: {modelPath.split('/')[-1]}")
    print(f"Inference time: {elapsedEval:.2f}s for {genEval.samples} images")
    print(f"Overall accuracy: {np.mean(yPredEval == yTrueEval)*100:.2f}%")
    print("\nClassification report:")
    print(classification_report(yTrueEval, yPredEval, target_names=labelsEval, digits=4))

    cmEval = confusion_matrix(yTrueEval, yPredEval)
    plt.figure(figsize=(6,6))
    sns.heatmap(cmEval, annot=True, fmt='d',
                xticklabels=labelsEval, yticklabels=labelsEval,
                cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

    print(f"Total parameters: {modelEval.count_params():,}")

evaluateModel('best_model_3class.keras', 'test_mini', imgSize=(224,224), batchSize=16)
evaluateModel('best_inception_model_3class.keras', 'test_miniInception', imgSize=(299,299), batchSize=16)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def getTestGeneratorUnique(testDirUnique, imgSizeUnique=(224,224), batchSizeUnique=16):
    testDataGenUnique = ImageDataGenerator(rescale=1./255)
    return testDataGenUnique.flow_from_directory(
        testDirUnique,
        target_size=imgSizeUnique,
        batch_size=batchSizeUnique,
        class_mode='categorical',
        shuffle=False
    )

def evaluateModelUnique(modelPathUnique, testDirUnique, imgSizeUnique=(224,224), batchSizeUnique=16):
    modelUnique = load_model(modelPathUnique)
    genUnique = getTestGeneratorUnique(testDirUnique, imgSizeUnique, batchSizeUnique)
    stepsUnique = int(np.ceil(genUnique.samples / batchSizeUnique))
    startUnique = time.time()
    predsUnique = modelUnique.predict(genUnique, steps=stepsUnique, verbose=0)
    elapsedUnique = time.time() - startUnique

    yTrueUnique = genUnique.classes
    yPredUnique = predsUnique.argmax(axis=1)
    labelsUnique = list(genUnique.class_indices.keys())

    print(f"\n>>> Model: {modelPathUnique.split('/')[-1]}")
    print(f"Inference time: {elapsedUnique:.2f}s for {genUnique.samples} images")
    print(f"Overall accuracy: {np.mean(yPredUnique == yTrueUnique)*100:.2f}%")
    print("\nClassification report:")
    print(classification_report(yTrueUnique, yPredUnique, target_names=labelsUnique, digits=4))

    cmUnique = confusion_matrix(yTrueUnique, yPredUnique)
    plt.figure(figsize=(6,6))
    sns.heatmap(cmUnique, annot=True, fmt='d',
                xticklabels=labelsUnique, yticklabels=labelsUnique,
                cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

    print(f"Total parameters: {modelUnique.count_params():,}")

evaluateModelUnique('best_model_11class.keras', 'test_mini', imgSizeUnique=(224,224), batchSizeUnique=16)
evaluateModelUnique('best_inception_model_11class.keras', 'test_miniInception', imgSizeUnique=(299,299), batchSizeUnique=16)